In [2]:
import configparser
from datetime import datetime, timedelta
import os
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format, dayofweek
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.types import DateType,StringType
from pyspark.sql import functions as F
from pyspark.sql.functions import avg
import pandas as pd

In [3]:
test_spark =  SparkSession.builder.getOrCreate()

## Load an example file from Jan 2012

In [4]:
temperatures_check_df = test_spark.read.parquet('parquet_file/temperatures/year=2010/month=2/part-00000-6a77b294-e335-47b9-a849-61dbbc798faf.c000.snappy.parquet')

In [5]:
temperatures_check_df.show()

+--------------+----------+--------+--------------------+-----------+--------+---------+------------+------------+
|temperature_id|      date|avg_temp|avg_temp_uncertainty|       city|latitude|longitude|airport_code|     city_id|
+--------------+----------+--------+--------------------+-----------+--------+---------+------------+------------+
|        369317|2010-02-01|   9.025|                0.26|Baton Rouge|  29.74N|   90.46W|     US-0321|154618822656|
|        369318|2010-02-01|   9.025|                0.26|Baton Rouge|  29.74N|   90.46W|        LA99|154618822656|
|        369319|2010-02-01|   9.025|                0.26|Baton Rouge|  29.74N|   90.46W|        LA69|154618822656|
|        369320|2010-02-01|   9.025|                0.26|Baton Rouge|  29.74N|   90.46W|        LA28|154618822656|
|        369321|2010-02-01|   9.025|                0.26|Baton Rouge|  29.74N|   90.46W|        LA05|154618822656|
|        369322|2010-02-01|   9.025|                0.26|Baton Rouge|  29.74N|  

In [6]:
temperatures_check_df.count()

186

In [7]:
cites_check_temp_df = temperatures_check_df.select(temperatures_check_df.city)

In [8]:
cites_check_temp_df = cites_check_temp_df.dropDuplicates(subset=['city'])

In [9]:
cites_check_temp_df.show()

+-----------+
|       city|
+-----------+
| Toms River|
|Baton Rouge|
|       Reno|
|    Jackson|
|    Spokane|
|Saint Louis|
|Minneapolis|
|     Denton|
| Cincinnati|
|   Syracuse|
+-----------+



## The I have the average temperature of Minneapolis, i want to find the population of it

In [66]:
demographics_check_df = test_spark.read.parquet('parquet_file/demographics/part-00021-fb4f5337-e7d5-45d8-90f3-84b91e39f144-c000.snappy.parquet')
demographics_check_df = demographics_check_df.filter(demographics_check_df.city_name == "Minneapolis")
demographics_check_df.show()

+------------+-----------+----------+----------+---------------+-----------------+----------------+------------+------------+-------------+----------+
|     city_id|  city_name|state_name|median_age|male_population|female_population|total_population|num_veterans|foreign_born|avg_household|state_code|
+------------+-----------+----------+----------+---------------+-----------------+----------------+------------+------------+-------------+----------+
|180388626445|Minneapolis| Minnesota|      32.4|         206547|           204388|          410935|       15217|       70769|         2.26|        MN|
+------------+-----------+----------+----------+---------------+-----------------+----------------+------------+------------+-------------+----------+



In [71]:
temperatures_check_df.createOrReplaceTempView("temperatures")

In [72]:
demographics_check_df.createOrReplaceTempView("demographics")

# HERE IS THE EXAMPLE QUERY FROM temperatures table and demographics table

## Find the average temperature in month 2 year 2010 and its population of Minneapolis

In [83]:
results_df =  test_spark.sql("SELECT distinct t.date, t.avg_temp, t.city, d.total_population, d.median_age FROM temperatures t JOIN demographics d on t.city_id = d.city_id WHERE t.city = 'Minneapolis'")

In [84]:
results_df.show()

+----------+--------+-----------+----------------+----------+
|      date|avg_temp|       city|total_population|median_age|
+----------+--------+-----------+----------------+----------+
|2010-02-01|   -9.07|Minneapolis|          410935|      32.4|
+----------+--------+-----------+----------------+----------+



# -9.07 average temperature but has a big number of total population
# Some other interesting analysis would be low average temperature has low population or nice average temperature has high foreign-born people

In [86]:
# HERE IS THE EXAMPLE QUERY FROM temperatures table and airport table

In [99]:
airport_check_df = test_spark.read.parquet('parquet_file/airports/state_code=MN/part-00001-87354b39-9502-4412-9050-46d33463cd64.c000.snappy.parquet')

In [100]:
airport_check_df.show()

+------------+-------------+--------------------+-------------------+
|airport_code|         type|                name|          city_name|
+------------+-------------+--------------------+-------------------+
|         SYN|small_airport|    Stanton Airfield|            Stanton|
|     US-0052|small_airport|      Joyner Airport|         Monticello|
|     US-0166|small_airport|     Shimpa Airstrip|             Euclid|
|     US-0316|     heliport|Appleton Municipa...|           Appleton|
|     US-0325|seaplane_base|Steam Boat Bay Se...|     East Gull Lake|
|     US-0327|     heliport|Lakewood Health C...|           Baudette|
|     US-0363|seaplane_base| Point Seaplane Base|              Orono|
|     US-0395|small_airport|   West Wind Airport|         Littlefork|
|     US-0438|     heliport|     Benson Heliport|             Benson|
|     US-0475|     heliport|International Fal...|International Falls|
|     US-0508|     heliport|Pipestone County ...|          Pipestone|
|     US-0509|small_

## Print if there are any heliport in Minneapolis, as well as its average temperature in month 2 year 2010 and its population

In [101]:
airport_check_df.createOrReplaceTempView("airports")

In [103]:
results_df =  test_spark.sql("SELECT distinct t.date, t.avg_temp, t.city, d.total_population, d.median_age, a.name FROM temperatures t JOIN demographics d on t.city_id = d.city_id JOIN airports a ON t.airport_code = a.airport_code WHERE t.city = 'Minneapolis' AND a.type='heliport'")

In [104]:
results_df.show()

+----------+--------+-----------+----------------+----------+--------------------+
|      date|avg_temp|       city|total_population|median_age|                name|
+----------+--------+-----------+----------------+----------+--------------------+
|2010-02-01|   -9.07|Minneapolis|          410935|      32.4|Hennepin County M...|
+----------+--------+-----------+----------------+----------+--------------------+



# So there is one heliport in the parquet file i read and i also have the average temperature and total population of the city

# I use partition by year and month in temperatures tables for easy for search for the date
# I use parition by state_code in airport tables for easy for search for the city in the state